# **6. Scaling**
---

Here we will:
- Create the scorecards.
- Generate the points map dictionary
- Predict the credit score from an input

## **6.1 Create Scorecards**
---

Assign score to each attribute by specifying:
- Odds of good of 30:1 at 300 points score, and
- 20 PDO (points to double the odds of good).

Thus, we can calculate the offset and factor:
- $\text{Factor}=\text{PDO}/ \ln(2)$
- $\text{Offset} = \text{Score} − {\text{Factor} ∗ \ln (\text{Odds of good})}$

In [1]:
# Import library
import pandas as pd
import numpy as np

# Load configuration
import src.utils as utils

Update the config file to define the references and dump the scorecards.

In [2]:
# Update the config file 
CONFIG_DATA = utils.config_load()
CONFIG_DATA

{'raw_dataset_path': 'data/raw/german_credit_data.csv',
 'dataset_path': 'data/output/data.pkl',
 'predictors_set_path': 'data/output/predictors.pkl',
 'response_set_path': 'data/output/response.pkl',
 'train_path': ['data/output/X_train.pkl', 'data/output/y_train.pkl'],
 'test_path': ['data/output/X_test.pkl', 'data/output/y_test.pkl'],
 'data_train_path': 'data/output/data_train.pkl',
 'data_train_binned_path': 'data/output/data_train_binned.pkl',
 'crosstab_list_path': 'data/output/crosstab_list.pkl',
 'WOE_table_path': 'data/output/WOE_table.pkl',
 'IV_table_path': 'data/output/IV_table.pkl',
 'WOE_map_dict_path': 'data/output/WOE_map_dict.pkl',
 'X_train_woe_path': 'data/output/X_train_woe.pkl',
 'response_variable': 'Risk',
 'test_size': 0.2,
 'num_variable': ['Age', 'Credit_amount', 'Duration'],
 'cat_variable': ['Sex',
  'Job',
  'Housing',
  'Saving_accounts',
  'Checking_account',
  'Purpose'],
 'missing_columns': ['Saving_accounts', 'Checking_account'],
 'num_of_bins': 4,
 '

In [3]:
# Function to convert the model's output into score points
def scaling():
    """Function to assign score points to each attribute"""

    # Define the references: score, odds, pdo
    pdo = CONFIG_DATA['pdo']
    score = CONFIG_DATA['score_ref']
    odds = CONFIG_DATA['odds_ref']

    # Load the best model
    best_model_path = CONFIG_DATA['best_model_path']
    best_model = utils.pickle_load(best_model_path)

    # Load the WOE table
    WOE_table_path = CONFIG_DATA['WOE_table_path']
    WOE_table = utils.pickle_load(WOE_table_path)

    # Load the best model's estimates table
    best_model_summary_path = CONFIG_DATA['best_model_summary_path']
    best_model_summary = utils.pickle_load(best_model_summary_path)

    # Calculate Factor and Offset
    factor = pdo/np.log(2)
    offset = score-(factor*np.log(odds))

    print('===================================================')
    print(f"Odds of good of {odds}:1 at {score} points score.")
    print(f"{pdo} PDO (points to double the odds of good).")
    print(f"Offset = {offset:.2f}")
    print(f"Factor = {factor:.2f}")
    print('===================================================')

    # Define n = number of characteristics
    n = best_model_summary.shape[0] - 1

    # Define b0
    b0 = best_model.intercept_[0]

    # Adjust characteristic name in best_model_summary_table
    num_cols = CONFIG_DATA['num_variable']
    for col in best_model_summary['Characteristic']:

        if col in num_cols:
            bin_col = col + '_bin'
        else:
            bin_col = col

        best_model_summary.replace(col, bin_col, inplace = True) 

    # Merge tables to get beta/parameter estimate for each characteristic
    scorecards = pd.merge(left = WOE_table,
                          right = best_model_summary,
                          how = 'left',
                          on = ['Characteristic']).dropna()
    
    # Define beta and WOE
    beta = scorecards['Estimate']
    WOE = scorecards['WOE']

    # Calculate the score point for each attribute
    scorecards['Points'] = round((offset/n) - factor*((b0/n) + (beta*WOE)))
    scorecards['Points'] = scorecards['Points'].astype('int')

    # Validate
    print('Scorecards table shape : ', scorecards.shape)
    
    # Dump the scorecards
    scorecards_path = CONFIG_DATA['scorecards_path']
    utils.pickle_dump(scorecards, scorecards_path)

    return scorecards

In [4]:
# Check the function
scaling()

Odds of good of 30:1 at 300 points score.
20 PDO (points to double the odds of good).
Offset = 201.86
Factor = 28.85
Scorecards table shape :  (31, 5)


,Characteristic,Attribute,WOE,Estimate,Points
0,Age_bin,"(18.999, 27.0]",-0.250071,-0.725105,24
1,Age_bin,"(27.0, 33.0]",-0.162248,-0.725105,25
2,Age_bin,"(33.0, 41.25]",0.433636,-0.725105,38
3,Age_bin,"(41.25, 75.0]",0.097164,-0.725105,31
8,Duration_bin,"(3.999, 12.0]",0.454913,-0.979380,42
9,Duration_bin,"(12.0, 18.0]",0.018868,-0.979380,29
10,Duration_bin,"(18.0, 24.0]",-0.030537,-0.979380,28
11,Duration_bin,"(24.0, 60.0]",-0.613683,-0.979380,12
12,Sex,female,-0.277765,-1.102988,20
13,Sex,male,0.127017,-1.102988,33


## **6.2 Predict the Credit Score**
---

Here we need to generate the points map dictionary to predit the credit score from an input.

Update the config file to dump the points map dictionary and the credit score.

In [5]:
# Update the config file 
CONFIG_DATA = utils.config_load()
CONFIG_DATA

{'raw_dataset_path': 'data/raw/german_credit_data.csv',
 'dataset_path': 'data/output/data.pkl',
 'predictors_set_path': 'data/output/predictors.pkl',
 'response_set_path': 'data/output/response.pkl',
 'train_path': ['data/output/X_train.pkl', 'data/output/y_train.pkl'],
 'test_path': ['data/output/X_test.pkl', 'data/output/y_test.pkl'],
 'data_train_path': 'data/output/data_train.pkl',
 'data_train_binned_path': 'data/output/data_train_binned.pkl',
 'crosstab_list_path': 'data/output/crosstab_list.pkl',
 'WOE_table_path': 'data/output/WOE_table.pkl',
 'IV_table_path': 'data/output/IV_table.pkl',
 'WOE_map_dict_path': 'data/output/WOE_map_dict.pkl',
 'X_train_woe_path': 'data/output/X_train_woe.pkl',
 'response_variable': 'Risk',
 'test_size': 0.2,
 'num_variable': ['Age', 'Credit_amount', 'Duration'],
 'cat_variable': ['Sex',
  'Job',
  'Housing',
  'Saving_accounts',
  'Checking_account',
  'Purpose'],
 'missing_columns': ['Saving_accounts', 'Checking_account'],
 'num_of_bins': 4,
 '

In [6]:
# Generate the Points map dict function
def get_points_map_dict():
    """Get the Points mapping dictionary"""
    # Load the Scorecards table
    scorecards = utils.pickle_load(CONFIG_DATA['scorecards_path'])

    # Initialize the dictionary
    points_map_dict = {}
    points_map_dict['Missing'] = {}
    unique_char = set(scorecards['Characteristic'])
    for char in unique_char:
        # Get the Attribute & WOE info for each characteristics
        current_data = (scorecards
                            [scorecards['Characteristic']==char]     # Filter based on characteristic
                            [['Attribute', 'Points']])                 # Then select the attribute & WOE
        
        # Get the mapping
        points_map_dict[char] = {}
        for idx in current_data.index:
            attribute = current_data.loc[idx, 'Attribute']
            points = current_data.loc[idx, 'Points']

            if attribute != 'Missing':
                points_map_dict[char][attribute] = points
                points_map_dict['Missing'][char] = np.nan
                
        if 'Missing' in current_data['Attribute'].tolist():
            points_map_dict['Missing'][char] = current_data.loc[current_data['Attribute']=='Missing', 'Points'].values[0]

    # Validate data
    print('Number of key : ', len(points_map_dict.keys()))

    # Dump
    utils.pickle_dump(points_map_dict, CONFIG_DATA['points_map_dict_path'])

    return points_map_dict
    

In [7]:
# Check the function
get_points_map_dict()

Number of key :  8


{'Missing': {'Purpose': nan,
  'Age_bin': nan,
  'Saving_accounts': 40,
  'Duration_bin': nan,
  'Checking_account': 60,
  'Sex': nan,
  'Job': nan},
 'Purpose': {'business': 27,
  'car': 26,
  'domestic appliances': 29,
  'education': 17,
  'furniture/equipment': 25,
  'radio/TV': 40,
  'repairs': 20,
  'vacation/others': 12},
 'Age_bin': {Interval(18.999, 27.0, closed='right'): 24,
  Interval(27.0, 33.0, closed='right'): 25,
  Interval(33.0, 41.25, closed='right'): 38,
  Interval(41.25, 75.0, closed='right'): 31},
 'Saving_accounts': {'little': 24,
  'moderate': 26,
  'quite rich': 48,
  'rich': 49},
 'Duration_bin': {Interval(3.999, 12.0, closed='right'): 42,
  Interval(12.0, 18.0, closed='right'): 29,
  Interval(18.0, 24.0, closed='right'): 28,
  Interval(24.0, 60.0, closed='right'): 12},
 'Checking_account': {'little': 5, 'moderate': 20, 'rich': 40},
 'Sex': {'female': 20, 'male': 33},
 'Job': {0: 27, 1: 31, 2: 29, 3: 26}}

Next, transform the raw input data into score points.

In [8]:
def transform_points(raw_data=None, type=None, CONFIG_DATA=None):
    """Replace data value with points"""
    # Load the numerical columns
    num_cols = CONFIG_DATA['num_variable']

    # Load the best model's estimates table
    best_model_summary_path = CONFIG_DATA['best_model_summary_path']
    best_model_summary = utils.pickle_load(best_model_summary_path)
    best_model_columns = best_model_summary['Characteristic'].tolist()[1:]

    # Load the points_map_dict
    points_map_dict = utils.pickle_load(CONFIG_DATA['points_map_dict_path'])

    # Load the saved data if type is not None
    if type is not None:
        raw_data = utils.pickle_load(CONFIG_DATA[f'{type}_path'][0])

    # Map the data
    points_data = raw_data.loc[:,best_model_columns].copy()

    for col in points_data.columns:
        # Perbaiki kolom numerik
        if col in num_cols:
            map_col = col + '_bin'
        else:
            map_col = col    

        points_data[col] = points_data[col].map(points_map_dict[map_col])

    # Map the data if there is a missing value or out of range value
    for col in points_data.columns:
        if col in num_cols:
            map_col = col + '_bin'
        else:
            map_col = col 

        points_data[col] = points_data[col].fillna(value=points_map_dict['Missing'][map_col])

    # Dump data
    if type is not None:
        utils.pickle_dump(points_data, CONFIG_DATA[f'X_{type}_points_path'])

    return points_data

In [9]:
# Check the function on the train set
X_train_points = transform_points(type='train', CONFIG_DATA=CONFIG_DATA)

X_train_points

,Age,Sex,Job,Saving_accounts,Checking_account,Duration,Purpose
485,31,33,26,24.0,20.0,42,26
390,25,33,26,24.0,60.0,29,26
23,31,33,29,26.0,20.0,42,26
814,31,33,29,24.0,5.0,12,26
107,25,33,29,24.0,20.0,42,26
...,...,...,...,...,...,...,...
324,38,20,29,24.0,60.0,29,26
428,24,33,29,24.0,60.0,42,25
637,24,33,29,24.0,60.0,12,40
688,38,33,29,26.0,60.0,42,40


Then, add a function to calculate the credit score.

Update the config file to dump the credit score.

In [10]:
# Update the config file 
CONFIG_DATA = utils.config_load()
CONFIG_DATA

{'raw_dataset_path': 'data/raw/german_credit_data.csv',
 'dataset_path': 'data/output/data.pkl',
 'predictors_set_path': 'data/output/predictors.pkl',
 'response_set_path': 'data/output/response.pkl',
 'train_path': ['data/output/X_train.pkl', 'data/output/y_train.pkl'],
 'test_path': ['data/output/X_test.pkl', 'data/output/y_test.pkl'],
 'data_train_path': 'data/output/data_train.pkl',
 'data_train_binned_path': 'data/output/data_train_binned.pkl',
 'crosstab_list_path': 'data/output/crosstab_list.pkl',
 'WOE_table_path': 'data/output/WOE_table.pkl',
 'IV_table_path': 'data/output/IV_table.pkl',
 'WOE_map_dict_path': 'data/output/WOE_map_dict.pkl',
 'X_train_woe_path': 'data/output/X_train_woe.pkl',
 'response_variable': 'Risk',
 'test_size': 0.2,
 'num_variable': ['Age', 'Credit_amount', 'Duration'],
 'cat_variable': ['Sex',
  'Job',
  'Housing',
  'Saving_accounts',
  'Checking_account',
  'Purpose'],
 'missing_columns': ['Saving_accounts', 'Checking_account'],
 'num_of_bins': 4,
 '

In [11]:
# Function to predict the credit score
def predict_score(raw_data, CONFIG_DATA):
    """Function to predict the credit score"""
    
    points = transform_points(raw_data = raw_data, 
                              type = None, 
                              CONFIG_DATA = CONFIG_DATA)
    
    score = int(points.sum(axis=1))

    # print(f"Credit Score : ", score)
    
    # cutoff_score = CONFIG_DATA['cutoff_score']

    # if score > cutoff_score:
    #     print("Recommendation : APPROVE")
    # else:
    #     print("Recommendation : REJECT")

    utils.pickle_dump(score, CONFIG_DATA['score_path'])

    return score


In [14]:
# Check the function with raw data input
tes_input = {
    'Age': 23,
    'Sex':'male',
    'Job':2,
    'Saving_accounts':'moderate',
    'Checking_account':'little',
    'Duration':6,
    'Purpose':'car'
}

tes = pd.DataFrame(tes_input, index=[0])

tes

,Age,Sex,Job,Saving_accounts,Checking_account,Duration,Purpose
0,23,male,2,moderate,little,6,car


In [15]:
# Predict the credit score
predict_score(raw_data=tes, CONFIG_DATA=CONFIG_DATA)

/var/folders/g0/kqn0dm690173t925jvn3s65h0000gn/T/ipykernel_57481/613627681.py:9: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  score = int(points.sum(axis=1))


185